# Sesión 12: Regresión Lineal Simple

### Importar paquetes requeridos

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
%matplotlib inline


<h2 id="understanding_data">Comprendiendo los datos</h2>

### `FuelConsumption.csv`:
Hemos descargado un conjunto de datos de consumo de combustible, **`FuelConsumption.csv`**, que contiene clasificaciones de consumo de combustible específicas del modelo y emisiones estimadas de dióxido de carbono para nuevos vehículos ligeros para la venta al por menor en Canadá. [Dataset source](http://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64)

- **MODELYEAR** e.g. 2014
- **MAKE** e.g. Acura
- **MODEL** e.g. ILX
- **VEHICLE CLASS** e.g. SUV
- **ENGINE SIZE** e.g. 4.7
- **CYLINDERS** e.g 6
- **TRANSMISSION** e.g. A6
- **FUEL CONSUMPTION in CITY(L/100 km)** e.g. 9.9
- **FUEL CONSUMPTION in HWY (L/100 km)** e.g. 8.9
- **FUEL CONSUMPTION COMB (L/100 km)** e.g. 9.2
- **CO2 EMISSIONS (g/km)** e.g. 182   --> low --> 0


<h2 id="reading_data">Leyendo los datos de entreada</h2>

In [ ]:
df = pd.read_csv("data/FuelConsumption.csv")

# take a look at the dataset
df.head()

<h2 id="data_exploration">Exploración de datos</h2>
Primero hagamos una exploración descriptiva de nuestros datos.

In [ ]:
# summarize the data
df.describe()

Vamos a seleccionar algunas funciones para explorar más:

In [ ]:
cdf = df[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB','CO2EMISSIONS']]
cdf.head(9)

Nosotros podemos graficar cada una de esas características:

In [ ]:
viz = cdf[['CYLINDERS','ENGINESIZE','CO2EMISSIONS','FUELCONSUMPTION_COMB']]
viz.hist()
plt.show()

Ahora, vamos a graficar cada una de las características vs la emisión, para observar cuán lineal es su relación:

In [ ]:
plt.scatter(cdf.FUELCONSUMPTION_COMB, cdf.CO2EMISSIONS,  color='blue')
plt.xlabel("FUELCONSUMPTION_COMB")
plt.ylabel("Emission")
plt.show()

In [ ]:
plt.scatter(cdf.ENGINESIZE, cdf.CO2EMISSIONS,  color='blue')
plt.xlabel("Engine size")
plt.ylabel("Emission")
plt.show()

#### Creando el conjunto de datos de entrenamiento y prueba
La división Train/Test Split implica dividir el conjunto de datos en conjuntos de entrenamiento y prueba, respectivamente, que son mutuamente excluyentes. Después de lo cual, entrena con el conjunto Train y prueba con el conjunto Test.
Esto proporcionará una evaluación más precisa de la precisión fuera de la muestra porque el conjunto de datos de prueba no es parte del conjunto de datos que se ha utilizado para entrenar los datos. Es más realista para problemas del mundo real.

Esto significa que conocemos el resultado de cada punto de datos en este conjunto de datos, ¡por lo que es genial probarlo! Y dado que estos datos no se han utilizado para entrenar el modelo, el modelo no tiene conocimiento del resultado de estos puntos de datos. Entonces, en esencia, es realmente una prueba fuera de muestra.

Dividamos nuestro conjunto de datos en conjuntos de train/test, el 80% de los datos completos para entrenamiento y el 20% para prueba. Creamos una máscara para seleccionar filas aleatorias usando la función __np.random.rand()__: 

In [ ]:
msk = np.random.rand(len(df)) < 0.8
train = cdf[msk]
test = cdf[~msk]

In [ ]:
len(train)

In [ ]:
len(test)

<h2 id="simple_regression">Modelo de regresión Simple</h2>
La regresión lineal se ajusta a un modelo lineal con coeficientes $\theta = (\theta_1, ..., \theta_n)$ para minimizar la 'suma residual de cuadrados' entre la variable independiente <b>x</b> y la variable dependiente <b>y</b> en el conjunto de datos por aproximación lineal.

#### Distribución de la data de entrenamiento

In [ ]:
plt.scatter(train.ENGINESIZE, train.CO2EMISSIONS,  color='blue')
plt.xlabel("Engine size")
plt.ylabel("Emission")
plt.show()

#### Modelando
Usando el paquete <b>sklearn</b> para modelar la data.

In [ ]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
train_x = np.asanyarray(train[['ENGINESIZE']])
train_y = np.asanyarray(train[['CO2EMISSIONS']])
regr.fit (train_x, train_y)
# The coefficients
print ('Coefficients: ', regr.coef_)
print ('Intercept: ',regr.intercept_)

CO2EMISSIONS = 125.90957529 + 39.03073028*ENGINESIZE

Como se mencionó anteriormente, __Coefficient__ e __Intercept__ en la regresión lineal simple, son los parámetros de la línea de ajuste.
Dado que es una regresión lineal simple, con solo 2 parámetros, y sabiendo que los parámetros son la intersección y la pendiente de la línea, sklearn puede estimarlos directamente a partir de nuestros datos.
Ten en cuenta que todos los datos deben estar disponibles para atravesar y calcular los parámetros.

#### Graficar salidas

Nosotros podemos graficar la línea de ajuste sobre los datos:

In [ ]:
plt.scatter(train.ENGINESIZE, train.CO2EMISSIONS,  color='blue')
plt.plot(train_x, regr.coef_[0][0]*train_x + regr.intercept_[0], '-r')
plt.xlabel("Engine size")
plt.ylabel("Emission")

#### Evaluación
comparamos los valores reales y los valores pronosticados para calcular la precisión de un modelo de regresión. Las métricas de evaluación proporcionan un papel clave en el desarrollo de un modelo, ya que proporcionan información sobre las áreas que requieren mejoras.

Existen diferentes métricas de evaluación del modelo, usamos MSE aquí para calcular la precisión de nuestro modelo en función del conjunto de pruebas: 
<ul>
    <li> Mean absolute error: Error absoluto medio es la media del valor absoluto de los errores. Esta es la métrica más fácil de entender, ya que es solo un error promedio.</li>
    <li> Mean Squared Error (MSE): Es la media del error cuadrado. Es más popular que el error absoluto medio porque el enfoque se orienta más hacia errores grandes. Esto se debe a que el término cuadrado aumenta exponencialmente los errores más grandes en comparación con los más pequeños.</li>
    <li> Root Mean Squared Error (RMSE): Esta es la raíz cuadrada del error cuadrático medio.</li>
    <li> R-squared no es un error, pero es una métrica popular para la precisión de tu modelo. Representa lo cerca que están los datos de la línea de regresión ajustada. Cuanto más alto sea el R cuadrado, mejor se ajustará el modelo a tus datos. La mejor puntuación posible es 1.0 y puede ser negativa (porque el modelo puede ser arbitrariamente peor).</li>
</ul>

In [ ]:
from sklearn.metrics import r2_score

test_x = np.asanyarray(test[['ENGINESIZE']])
test_y = np.asanyarray(test[['CO2EMISSIONS']])
test_y_hat = regr.predict(test_x)

print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_hat - test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((test_y_hat - test_y) ** 2))
print("R2-score: %.2f" % r2_score(test_y_hat , test_y) )